# TensorFlow to OpenVINO

This short tutorial shows how to convert a TensorFlow MobilenetV3 model to OpenVINO IR format.
Model documentation: https://docs.openvinotoolkit.org/latest/omz_models_model_mobilenet_v3_small_1_0_224_tf.html

### Imports

In [ ]:
import json
import sys
import time
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import mo_tf
import numpy as np
from openvino.inference_engine import IECore

### Settings

In [ ]:
# The paths of the source and converted models
model_path = Path("model/v3-small_224_1.0_float.pb")
ir_path = Path(model_path).with_suffix(".xml")

## Convert the Model to OpenVINO IR Format

### Convert TensorFlow model to OpenVINO IR Format

Call the OpenVINO Model Optimizer tool to convert the TensorFlow model to OpenVINO IR, with FP16 precision. The models are saved to the current directory. We add the mean values to the model and scale the output with the standard deviation with `--scale_values`. With these options, it is not necessary to normalize input data before propagating it through the network. The original model expects input images in RGB format. The converted model also expects images in RGB format. If you want the converted model to work with BGR images, you can use the `--reverse-input-channels` option. See the [Model Optimizer Developer Guide](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) for more information about Model Optimizer, including a description of the command line options. Check the [model documentation](https://docs.openvinotoolkit.org/latest/omz_models_model_mobilenet_v3_small_1_0_224_tf.html) for information about the model, including input shape, expected color order and mean values. 

We first construct the command for Model Optimizer, and then execute this command in the notebook by prepending the command with a `!`. There may be some errors or warnings in the output. Model Optimization was succesful if the last lines of the output include `[ SUCCESS ] Generated IR version 10 model.
`

In [ ]:
# Get the path to the Model Optimizer script
mo_path = str(Path(mo_tf.__file__))

# Construct the command for Model Optimizer
mo_command = f""""{sys.executable}"
                 "{mo_path}" 
                 --input_model "{model_path}" 
                 --input_shape "[1,224,224,3]" 
                 --mean_values="[127.5,127.5,127.5]"
                 --scale_values="[127.5]" 
                 --data_type FP16 
                 --output_dir "{model_path.parent}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert TensorFlow to OpenVINO:")
print(mo_command)

In [ ]:
# Run Model Optimizer if the IR model file does not exist
if not ir_path.exists():
    print("Exporting TensorFlow model to IR... This may take a few minutes.")
    ! $mo_command
else:
    print(f"IR model {ir_path} already exists.")

## Test Inference on the Converted Model

### Load the model

In [ ]:
ie = IECore()
net = ie.read_network(str(ir_path), str(ir_path.with_suffix(".bin")))
exec_net = ie.load_network(net, "CPU")

### Get model information

In [ ]:
input_key = list(exec_net.input_info)[0]
output_key = list(exec_net.outputs.keys())[0]
network_input_shape = exec_net.input_info[input_key].tensor_desc.dims

### Load image

Load an image, resize it, and convert it to network input shape

In [ ]:
image = cv2.cvtColor(cv2.imread("data/coco.jpg"), cv2.COLOR_BGR2RGB)
# Resize image to network input image shape
resized_image = cv2.resize(image, (224, 224))
# Transpose image to network input shape
input_image = np.reshape(resized_image, network_input_shape) / 255
input_image = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0)
plt.imshow(image)

### Do inference

In [ ]:
result = exec_net.infer(inputs={input_key: input_image})[output_key]
result_index = np.argmax(result)

In [ ]:
imagenet_classes = json.loads(open("utils/imagenet_class_index.json").read())
imagenet_classes = {
    int(key) + 1: value for key, value in imagenet_classes.items()
}
imagenet_classes[result_index]

## Timing

Measure the time it takes to do inference on thousand images. This gives an indication of performance. For more accurate benchmarking, use the [OpenVINO benchmark tool](https://github.com/openvinotoolkit/openvino/tree/master/inference-engine/tools/benchmark_tool). Note that many optimizations are possible to improve the performance. 

In [ ]:
num_images = 1000
start = time.perf_counter()
for _ in range(num_images):
    exec_net.infer(inputs={input_key: input_image})
end = time.perf_counter()
time_ir = end - start
print(
    f"IR model in Inference Engine/CPU: {time_ir/num_images:.4f} "
    f"seconds per image, FPS: {num_images/time_ir:.2f}"
)